## Image Classification

### Download dependencies

In [114]:
# pip install numpy

In [115]:
# pip uninstall torch torchvision torchaudio -y

In [116]:
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/rocm6.2.4

In [117]:
# pip install torch torchvision torchaudio

In [118]:
# pip install tqdm

In [119]:
import numpy as np
from PIL import Image
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split

In [120]:
print(torch.cuda.is_available())

False


In [121]:
transform = transforms.Compose([
    transforms.Resize(256),            
    transforms.CenterCrop(224), 
    transforms.RandomHorizontalFlip(p=0.5),  # Randomly flip images
    transforms.RandomRotation(20),  # Rotate images by up to 20 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1)), # Slight translations
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])


In [122]:
dataset = torchvision.datasets.ImageFolder(root='dataset/Images', transform=transform)

train_size = int(0.9 * len(dataset))
test_size = len(dataset) - train_size

train_data, test_data = random_split(dataset, [train_size, test_size])

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True, num_workers=2)

In [123]:
image, label = train_data[0]

In [124]:
image.size()

torch.Size([3, 224, 224])

In [125]:
class_name = ['Action','Horror','Romance','Sci-Fi','Sports']

In [126]:
class NeuralNet(nn.Module):

    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 12, 5) # (12, 220, 220)
        self.pool = nn.MaxPool2d(2, 2) # (12, 110, 110)
        self.conv2 = nn.Conv2d(12, 24, 5) # (24, 106, 106) -> (24, 53, 53)

        self.fc1 = nn.Linear(24 * 53 * 53, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  
        x = self.pool(F.relu(self.conv2(x)))  
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))         
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x



In [127]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = net.to(device)

In [ ]:
# from tqdm import tqdm

# num_of_epochs = 800

# for epoch in range(num_of_epochs):
    
#     running_loss = 0.0
#     for i, data in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}")):
        
#         inputs, labels = data
#         inputs, labels = inputs.to(device), labels.to(device)

#         optimizer.zero_grad()
#         outputs = net(inputs)
#         loss = loss_function(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()

#     avg_loss = running_loss / len(train_loader)

#     if (epoch + 1) % 20 == 0:
#         print(f"Epoch {epoch+1}/{num_of_epochs} - Loss: {avg_loss:.4f}")


In [129]:
# torch.save(net.state_dict(), 'models/trained_image_model_SGDV4.pth')

In [130]:
net = NeuralNet()
net.load_state_dict(torch.load('models/trained_image_model_SGDV3.pth', map_location=torch.device('cpu')))

<All keys matched successfully>

In [131]:
test_dataset = torchvision.datasets.ImageFolder(root="dataset/Test_Images", transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

In [132]:
correct = 0
total = 0

net.eval()

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs,1) 
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy: {accuracy:.2f}%")

Accuracy: 43.00%


In [141]:
new_transform = transforms.Compose([
    transforms.Resize(256),            
    transforms.CenterCrop(224), 
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

def load_image(image_path):
    image = Image.open(image_path)
    image = new_transform(image)
    image = image.unsqueeze(0)
    return image

image_paths = ['dataset/Test_Images/Sports/6.jpg']

images = [load_image(img) for img in image_paths]

net.eval()
with torch.no_grad():
    for image in images:
        output = net(image)
        _, predicted = torch.max(output, 1)
        print(f"Prediction: {class_name[predicted.item()]}")

Prediction: Sports
